In [23]:
import os
import pandas as pd

In [24]:
PATH = os.path.join("../data/Hongkong.csv")
pd.read_csv(PATH)

,YEAR,Hong Kong/CGPD RELATIVE TO THE UNITED STATES (unit US=100),Hong Kong/CONSUMPTION SHARE OF CGPD (unit %),Hong Kong/CONSUMPTION SHARE OF RGDPL (unit %),Hong Kong/CURRENT SAVINGS (unit %),Hong Kong/EXCHANGE RATE (unit US=1),Hong Kong/GOVERNMENT SHARE OF CGDP (unit %),Hong Kong/GOVERNMENT SHARE OF RGDPL (unit %),Hong Kong/INVESTMENT SHARE OF CGDP (unit %),Hong Kong/INVESTMENT SHARE OF RGDPL (unit %),Hong Kong/OPENNESS IN CONSTANT PRICES (unit %),Hong Kong/OPENNESS IN CURRENT PRICES (unit %),Hong Kong/POPULATION (unit 000S),Hong Kong/PRICE LEVEL OF CONSUMPTION (unit PPP/XRATE),Hong Kong/PRICE LEVEL OF GOVERNMENT (unit PPP/XRATE),Hong Kong/PRICE LEVEL OF GROSS DOMESTIC PRODUCT (unit US=100),Hong Kong/PRICE LEVEL OF INVESTMENT (unit PPP/XRATE),Hong Kong/RATIO OF GNP TO GDP (CURRENT PRICES) (unit %),Hong Kong/REAL GROSS DOMESTIC PRODUCT PER CAPITA (CURRENT PRICES) (unit $ CURRENT)
0,1950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
